### Synthetic drift detection

In [ ]:
import sys
import pandas as pd
import numpy as np
import os
from functools import reduce
import datetime
import matplotlib.pyplot as plt

sys.path.append("../..")

from gemini.constants import *
from gemini.utils import *
from drift_detector.plotter import plot_drift_samples_pval, errorfill, plot_roc, plot_pr, linestyles, markers, colors, brightness, colorscale
from drift_detector.utils import scale
from drift_detector.detector import Detector
from drift_detector.reductor import Reductor
from drift_detector.tester import TSTester, DCTester
from drift_detector.experimenter import Experimenter
from drift_detector.synthetic_applicator import SyntheticShiftApplicator, apply_predefined_shift

## Parameters ##

In [ ]:
PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/"
MODEL_PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/random/random_lstm.pt"
TIMESTEPS = 6
AGGREGATION_TYPE = "time_flatten"
CONTEXT_TYPE="lstm"
REPRESENTATION="rf"
HOSPITALS = ["SMH", "MSH", "THPC", "THPM", "UHNTG", "UHNTW", "PMH"]
OUTCOME = "mortality"

## Query data

In [ ]:
admin_data, x, y = get_gemini_data(PATH)

## Preprocess data

In [ ]:
# Get subset
(X_tr, y_tr), (X_val, y_val), (X_t, y_t), feats, admin_data = import_dataset_hospital(admin_data, x, y, "random", OUTCOME, HOSPITALS)

# Normalize data
X_tr_normalized  = normalize(AGGREGATION_TYPE, admin_data, TIMESTEPS, X_tr)
X_val_normalized = normalize(AGGREGATION_TYPE, admin_data, TIMESTEPS, X_val)
X_t_normalized = normalize(AGGREGATION_TYPE, admin_data, TIMESTEPS, X_t)

# Get labels
y_tr = get_label(admin_data, X_tr, "mortality")
y_val = get_label(admin_data, X_val, "mortality")
y_t = get_label(admin_data, X_t, "mortality")
    
# Scale data
X_tr_scaled = scale(X_tr_normalized)
X_val_scaled = scale(X_val_normalized)
X_t_scaled = scale(X_t_normalized)

# Process data
X_tr_final = process(AGGREGATION_TYPE, TIMESTEPS, X_tr_scaled)
X_val_final = process(AGGREGATION_TYPE, TIMESTEPS, X_val_scaled)
X_t_final = process(AGGREGATION_TYPE, TIMESTEPS, X_t_scaled)

## Dimensionality reduction ##

In [ ]:
DR_TECHNIQUE = input("Select dimensionality reduction technique: ")

reductor = Reductor(
    dr_method = DR_TECHNIQUE,
    model_path = MODEL_PATH,
    var_ret = 0.8,
)
reductor.fit(X_tr_final)

## Tester ##

In [ ]:
MD_TEST = input("Select test method: ")

tester = TSTester(
    tester_method=MD_TEST,
)

## Detector ##

In [ ]:
detector = Detector(
    reductor = reductor,
    tester = tester,
    p_val_threshold = 0.05,
)

## SyntheticShiftApplicator

In [ ]:
syntheticshiftapplicator = SyntheticShiftApplicator(
    shift_type = "gn_shift",
) 

## Run custom experiments

In [ ]:
experimenter_custom = Experimenter(
    detector = detector,
    syntheticshiftapplicator = syntheticshiftapplicator,
    admin_data = admin_data
)

X_t_final_shifted = experimenter_custom.apply_synthetic_shift(
    X_t_final,
    shift_type="gn_shift",
    delta=0.01,
    noise_amt=0.01,
    clip=False
)

results = experimenter_custom.detect_shift_samples(
        X_val_final, 
        X_t_final_shifted
    )

results

## Run predefined shift experiment

In [ ]:
SHIFT = input("Select shift experiment: ")

if SHIFT =='ko_shift':
    shifts = ['ko_shift_0.1','ko_shift_0.5','ko_shift_1.0']
elif SHIFT == 'small_gn_shift':
    shifts = ['small_gn_shift_0.1','small_gn_shift_0.5','small_gn_shift_1.0']
elif SHIFT == 'medium_gn_shift':
    shifts = ['medium_gn_shift_0.1', 'medium_gn_shift_0.5','medium_gn_shift_1.0']
elif SHIFT == 'large_gn_shift':
    shifts = ['large_gn_shift_0.1','large_gn_shift_0.5','large_gn_shift_1.0']
elif SHIFT == 'mfa_shift':
    shifts = ['mfa_shift_0.25','mfa_shift_0.5','mfa_shift_0.75']
elif SHIFT == 'cp_shift':
    shifts = ['cp_shift_0.25','cp_shift_0.75']
elif SHIFT == 'small_bn_shift':
    shifts = ['small_bn_shift_0.1','small_bn_shift_0.5','small_bn_shift_1.0']
elif SHIFT == 'medium_bn_shift':
    shifts = ['medium_bn_shift_0.1','medium_bn_shift_0.5','medium_bn_shift_1.0']
elif SHIFT == 'large_bn_shift':
    shifts = ['large_bn_shift_0.1','large_bn_shift_0.5','large_bn_shift_1.0']

In [ ]:
experimenter_predefined = Experimenter(
    detector = detector,
    admin_data=admin_data
)

shift_results = {}
for si, shift in enumerate(shifts):
    X_t_final_shifted = X_t_final.copy()
    X_t_final_shifted, _ = apply_predefined_shift(shift, X=X_t_final_shifted,y=y_t)
    results = experimenter_predefined.detect_shift_samples(
        X_val_final, 
        X_t_final_shifted
    )
    shift_results.update({shift:results}) 

In [ ]:
experimenter_predefined.detect_shift_sample(
        X_val_final, 
        X_t_final_shifted,
        sample=100
)

## Plot shift experiments

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))
plt.hist(X_val_final[:,0], bins=50, alpha=0.5,label='val',density=True)
plt.hist(X_t_final[:,0], bins=50, alpha=0.5,label='test',density=True)
plt.hist(X_t_final_shifted[:,0], bins=50, alpha=0.5 , label='test+noise',density=True)
fig.legend(loc='upper right')
plt.show()

In [ ]:
plot_drift_samples_pval(shift_results, 0.05)